### 프로그램 설치 

In [1]:
! pip install konlpy

In [2]:
! pip install moviepy

In [3]:
!pip install pytube

In [4]:
!pip install kss

In [5]:
!pip install speechRecognition

In [6]:
!pip install moviepy

In [7]:
!pip install konlpy

In [8]:
!pip install regex

### 라이브러리 및 모델 불러오기 

In [9]:
from flask import Flask ,render_template
from flask import request, redirect
from konlpy.tag import Kkma,Okt, Twitter, Komoran # 형태소 분석 라이브러리
from moviepy.editor import * # 영상을 오디오 파일로 변환
from moviepy.editor import VideoFileClip, concatenate_videoclips
import moviepy.editor as mp
from pytube import YouTube # 유튜브 영상 다운로드 또는 불러오기
import pytube
import tqdm as tq

import kss # 텍스트 문장으로 바꾸는 라이브러리
import speech_recognition as sr # 오디오 파일 또는 음성을 텍스트로 변환
import pandas as pd
# BOW = BAG of WORD : 단어가방, 단어모음, 단어사전
from sklearn.feature_extraction.text import CountVectorizer
# 위 도구는 빈도수 기반 벡터화 도구

### 알고리즘 시작

### 영상 -> 음성 -> 텍스트추출

In [10]:
# https://www.youtube.com/watch?v=kFnHWpGs-18  :: 스마트 인재 개발원
# https://www.youtube.com/watch?v=lZi3k_GzfCk  :: 서강대 2:25 ~ 4: 10
youtube=input('다운로드 받을 유튜브 영상 링크 : ')
yt = pytube.YouTube(youtube)
title = yt.title

다운로드 받을 유튜브 영상 링크 : https://www.youtube.com/watch?v=lZi3k_GzfCk


In [11]:
# 영상 다운로드 경로

stream = yt.streams.all()[0]
stream.download(output_path='C:/Users/smhrd/Desktop/project/download') 

# 영상 오디오 파일로 변환 
clip = mp.VideoFileClip("download/python 02.3gpp")
newsound = clip.subclip("00:02:25","00:04:10") # 시간지정된 오디오 저장 
newsound.audio.write_audiofile("download/audio5.wav",16000,2,2000,'pcm_s16le')

# 오디오 파일 로드
filename = "download/audio5.wav"

# 오디오 파일 텍스트 추출
text = []
r = sr.Recognizer()
with sr.AudioFile(filename) as source:
    audio_data = r.record(source)
    text = r.recognize_google(audio_data,language='ko-KR')
    # print(text)

[Korean Sentence Splitter]: C:\Users\smhrd\anaconda3\envs\deep\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated function all (This object can be treated as a list, all() is useless).
  This is separate from the ipykernel package so we can avoid doing imports until



MoviePy - Writing audio in download/audio5.wav


MoviePy - Done.


### 텍스트 형태소 분리

In [12]:
# 형태소 구분하는 함수
# - 사용해야할 품사가 생각보다 많음 ( komoran 기준 ) 
#   - 명사 NN -> 일반명사 NNG // 고유명사 NNP // 의존명사 NNB

def lemmatize(word):
    morphtags = Komoran().pos(word)
    if morphtags[0][1] == 'NNG' or morphtags[0][1] == 'NNP': 
        return morphtags[0][0]

In [13]:
# kss 활용 텍스트 문장 화
word_list = kss.split_sentences(text)
word_list

[Korean Sentence Splitter]: Initializing Pynori...


['저를 먼저 알아보겠습니다',
 '컴퓨터는 키보드 마우스 와 같은 입력장치와 모니터 프린터와 같은 출력장치가 있습니다',
 '컴퓨터 내부의 주요 부분으로는 중앙처리장치와 메모리가 있습니다',
 '중앙처리장치는 CPU라고 하는데 프로그램 수행에 있어서']

In [14]:
# 명사만 가져오기 위한 삭제
okt = Okt()
headline = []
stopwords = [ '의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','등','으로도']
for sentence in word_list:
    temp = []
    # morphs() : 형태소 단위로 토큰화
    # stem = True : 형태소에서 어간을 추출
    temp = okt.morphs(sentence, stem = True)
    temp = [word for word in temp if not word in stopwords]
    headline.append(temp)

In [15]:
# konlpy 트위터 이용 형태소 분류
twitter = Twitter()
sentences_tag = []
for word in headline:
    for i in word :
        morph = twitter.pos(i)
        sentences_tag.append(morph)
# print(sentences_tag)

[Korean Sentence Splitter]: C:\Users\smhrd\anaconda3\envs\deep\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')



In [16]:
#  형태소 분류
noun_adj_list=[]
for i1 in sentences_tag:
    for word, tag in i1:
        if tag in ['Noun','Verb','Number','Adjective','Adverb']:
            noun_adj_list.append(word)
# print(noun_adj_list)
# 형태소 분류
for i in range(len(noun_adj_list)):
    #print(lemmatize(noun_adj_list[i]))
    if lemmatize(noun_adj_list[i]) != None :
        noun_adj_list[i] = lemmatize(noun_adj_list[i])
        print(noun_adj_list)

arr_list = noun_adj_list
print(arr_list)

['저', '먼저', '알아보다', '컴퓨터', '키', '보드', '마우스', '오다', '같다', '입력장치', '모니터', '프린터', '같다', '출력장치', '있다', '컴퓨터', '내부', '주요', '부분', '중앙', '처리장치', '메모리', '있다', '중앙', '처리장치', '하다', '프로그램', '수행', '있다']
['저', '먼저', '알아보다', '컴퓨터', '키', '보드', '마우스', '오다', '같다', '입력장치', '모니터', '프린터', '같다', '출력장치', '있다', '컴퓨터', '내부', '주요', '부분', '중앙', '처리장치', '메모리', '있다', '중앙', '처리장치', '하다', '프로그램', '수행', '있다']
['저', '먼저', '알아보다', '컴퓨터', '키', '보드', '마우스', '오다', '같다', '입력장치', '모니터', '프린터', '같다', '출력장치', '있다', '컴퓨터', '내부', '주요', '부분', '중앙', '처리장치', '메모리', '있다', '중앙', '처리장치', '하다', '프로그램', '수행', '있다']
['저', '먼저', '알아보다', '컴퓨터', '키', '보드', '마우스', '오다', '같다', '입력장치', '모니터', '프린터', '같다', '출력장치', '있다', '컴퓨터', '내부', '주요', '부분', '중앙', '처리장치', '메모리', '있다', '중앙', '처리장치', '하다', '프로그램', '수행', '있다']
['저', '먼저', '알아보다', '컴퓨터', '키', '보드', '마우스', '오다', '같다', '입력', '모니터', '프린터', '같다', '출력장치', '있다', '컴퓨터', '내부', '주요', '부분', '중앙', '처리장치', '메모리', '있다', '중앙', '처리장치', '하다', '프로그램', '수행', '있다']
['저', '먼저', '알아보다', '컴퓨터', '키', '보드', '마우스', '오

### 단어-영상 합치기

In [17]:
# 영상 합치기
    
clips = []
for i in range(len(arr_list)):
    # print("*****", clips)
    try:
        # arr_list = ['보고', '강의', '목표', '대해', '이야기', '하다', '다음', '수업', '부터', '본격', '적', '파이썬', '언어', '대한', '학습', '시작', '하다', '수', '있다', '파이썬', '언어', '설치', '하다', '과정', '보다', '교육', '점점', '중요하다', '이유', '소프트웨어', '미래', '사회', '요구', '하다', '용량', '속초', '발전']
        mov = VideoFileClip("Data_Deep/30개영상/"+arr_list[i]+".mp4")
        clips.append(mov)
        print("try")
    except:
        print('skip')
        
    print("last", clips)
final_clip = concatenate_videoclips(clips, method='compose') # concat함수를 이용해 비디오를 합치기
final_clip.write_videofile("Success/sua5.mp4")

skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []
skip
last []


ValueError: max() arg is an empty sequence

## 테스트

In [ ]:
# 영상 다운로드 경로
stream = yt.streams.all()[0]
stream.download(output_path='C:/Users/smhrd/Desktop/project/download') 

# 영상 오디오 파일로 변환 
clip = mp.VideoFileClip("download/python 02.3gpp")
newsound = clip.subclip("00:01:10","00:01:30") # 20 sec
newsound.audio.write_audiofile("download/audio5.wav",16000,2,2000,'pcm_s16le')

# 오디오 파일 로드
filename = "download/audio5.wav"

In [ ]:
# 오디오 파일 텍스트 추출
text = []
r = sr.Recognizer()
with sr.AudioFile(filename) as source:
    audio_data = r.record(source)
    text = r.recognize_google(audio_data,language='ko-KR')
    # print(text)

# kss 활용 텍스트 문장 화
word_list = kss.split_sentences(text)
word_list

## 영상 합치기 테스트

In [82]:
# 라벨 + 영상 합치기
import  pandas as pd 
data = pd.read_csv('word_data.csv',encoding='euc-kr')
data[:3]

,num,word,videoname,att,start,end,filename
0,1,고민,NIA_SL_WORD0001_REAL01_F.mp4,NaN,1.743,3.103,NIA_SL_WORD0001_REAL01_F_morpheme.json
1,2,뻔뻔,NIA_SL_WORD0002_REAL01_F.mp4,NaN,1.831,3.361,NIA_SL_WORD0002_REAL01_F_morpheme.json
2,3,수어,NIA_SL_WORD0003_REAL01_F.mp4,NaN,1.052,2.138,NIA_SL_WORD0003_REAL01_F_morpheme.json


In [83]:
word_list = ['고민','뻔뻔','수어']
word_list

['고민', '뻔뻔', '수어']

In [87]:
data.iloc[:2,2]

0    NIA_SL_WORD0001_REAL01_F.mp4
1    NIA_SL_WORD0002_REAL01_F.mp4
Name: videoname, dtype: object

### 리스트에 담긴 단어 + 영상찾기 + 시작점 끝점 자르고 병합

In [88]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

In [115]:
clips = []
#  리스트 속 단어 == DataFrame 속 단어, 영상 찾아서 clips 에 추가 
for i in range(len(word_list)): # 
    for j in range(len(word_list)):
        if (word_list[i]==data.iloc[j,1]) :
            print(data.iloc[j,2]) # videoname 으로 영상 찾음 + 시작점,끝점 자름 
            mov = VideoFileClip('3000_word/'+data.iloc[j,2]).subclip(data.iloc[j,4],data.iloc[j,5])
            clips.append(mov) 
        else :
            pass
# concat함수를 이용해 비디오를 합치기    
final_clip = concatenate_videoclips(clips, method='compose')
final_clip.write_videofile("Success/sua.mp4")   

NIA_SL_WORD0001_REAL01_F.mp4
NIA_SL_WORD0002_REAL01_F.mp4
NIA_SL_WORD0003_REAL01_F.mp4
Moviepy - Building video Success/sua5.mp4.
Moviepy - Writing video Success/sua5.mp4



Moviepy - Done !
Moviepy - video ready Success/sua5.mp4


## 문장 데이터 테스트

In [29]:
text = ['d','d']

In [39]:
word_list = ['입력된 데이터는 숫자인 경우에는 수치 데이터라고 하고 문자인 경우에는 문자데이터라고 합니다','다시 말하면 입력된 데이터는 각자 데이터 유형을 갖게 되는데 이것을 자료형이라고 합니다', 
    '프로그램을 수행하면 그 프로그램이 다루는 모든 데이터는 메모리 위에 저장이 되어야 한다고 했습니다', 
    '그때 어떤 식으로 저장이 되는지에 대해서 설명하겠습니다', 
    '메모리에 저장된 자료를 우리는 객체라고 부릅니다', 
    '모든 객체는 자료형을 갖는데 수치자료의 경우에는 정수냐 실수냐로 구분을하고, 또는 문자 데이터가 올라가기도 합니다', 
    '그래서 각각의 대해서 우리가 자료를 갖다 붙이게 되고 모든 개체는 고유 ID를 갖도록 합니다', 
    '그리고 객체를 저장하는 공간을 변수라고 하고 그 공간에 이름을 변수명이라고 해서 그 이름을 통해서 객체 접근 할 수가 있는데 간단하게 아래에서 객체와 변수를 이 해해 보도록 하겠습니다', 
    '데이터를 만들려면 즉 객체를 만들려면 파이썬에서는 어떤 이름은 하고서 값을 적어 줘야 합니다', 
    '예를 들어서 에이는 100이라고 한다면 에이가 백을 갖도록 한다는 뜻입니다', 
    '는(=)은 같다는 뜻이 아니라 100이라는 값을 에이에 넣겠다는 이야기가 됩니다', 
    '그래서 100이라는 값이 에이에 들어가게됩니다', 
    '이때 내부적으로 일어나는 일은 100이라는 데이터가 생겨서 그 100이라는 데이터를 객체라고 부르고요 객체가 저장이 되는데 저장이 되는 공간을 우리는 변수라고 부릅니다', 
    '그리고이 빨간 박스가 바로 변수가 되는데요 이 변수에는 이름이 하나씩 가서 붙게 됩니다', 
    '그 이름을 변수명이라고 하고 여기에서는 변수명이 A가 붙은 예제입니다', 
    '그래서 다음부터 이 100이라는 데이터를 가지고 오려면 변수명 에이를 이용을 해서 가져오게 됩니다']

In [40]:
# 형태소 구분하는 함수
# - 사용해야할 품사가 생각보다 많음 ( komoran 기준 ) 
#   - 명사 NN -> 일반명사 NNG // 고유명사 NNP // 의존명사 NNB

def lemmatize(word):
    morphtags = Komoran().pos(word)
    if morphtags[0][1] == 'NNG' or morphtags[0][1] == 'NNP': 
        return morphtags[0][0]

In [41]:
# 명사만 가져오기 위한 삭제
okt = Okt()
headline = []
stopwords = [ '된','의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','등','으로도']
for sentence in word_list:
    temp = []
    # morphs() : 형태소 단위로 토큰화
    # stem = True : 형태소에서 어간을 추출
    temp = okt.morphs(sentence, stem = True)
    temp = [word for word in temp if not word in stopwords]
    headline.append(temp)

In [42]:
# konlpy 트위터 이용 형태소 분류
twitter = Twitter()
sentences_tag = []
for word in headline:
    for i in word :
        morph = twitter.pos(i)
        sentences_tag.append(morph)
# print(sentences_tag)

[[('입력', 'Noun')], [('되다', 'Verb')], [('데이터', 'Noun')], [('숫자', 'Noun')], [('인', 'Noun')], [('경우', 'Noun')], [('에는', 'Josa')], [('수치', 'Noun')], [('데이터', 'Noun')], [('라고', 'Josa')], [('하다', 'Verb')], [('문자', 'Noun')], [('인', 'Noun')], [('경우', 'Noun')], [('에는', 'Josa')], [('문자', 'Noun')], [('데이터', 'Noun')], [('라고', 'Josa')], [('하다', 'Verb')], [('다시', 'Noun')], [('말', 'Noun')], [('하다', 'Verb')], [('입력', 'Noun')], [('되다', 'Verb')], [('데이터', 'Noun')], [('각자', 'Noun')], [('데이터', 'Noun')], [('유형', 'Noun')], [('을', 'Josa')], [('갖다', 'Verb')], [('되다', 'Verb')], [('것', 'Noun')], [('을', 'Josa')], [('자료형', 'Noun')], [('이라고', 'Josa')], [('하다', 'Verb')], [('프로그램', 'Noun')], [('을', 'Josa')], [('수행', 'Noun')], [('하다', 'Verb')], [('그', 'Noun')], [('프로그램', 'Noun')], [('다루다', 'Verb')], [('모든', 'Noun')], [('데이터', 'Noun')], [('메모리', 'Noun')], [('위', 'Noun')], [('저장', 'Noun')], [('되어다', 'Verb')], [('하다', 'Verb')], [('하다', 'Verb')], [('그때', 'Noun')], [('어떻다', 'Adjective')], [('식', 'Noun')], [('저장', 'Noun')]

In [43]:
#  형태소 분류
noun_adj_list=[]
for i1 in sentences_tag:
    for word, tag in i1:
        if tag in ['Noun','Verb','Number','Adjective','Adverb']:
            noun_adj_list.append(word)
# print(noun_adj_list)
# 형태소 분류
for i in range(len(noun_adj_list)):
    #print(lemmatize(noun_adj_list[i]))
    if lemmatize(noun_adj_list[i]) != None :
        noun_adj_list[i] = lemmatize(noun_adj_list[i])
        print(noun_adj_list)

arr_list = noun_adj_list

['입력', '되다', '데이터', '숫자', '인', '경우', '수치', '데이터', '하다', '문자', '인', '경우', '문자', '데이터', '하다', '다시', '말', '하다', '입력', '되다', '데이터', '각자', '데이터', '유형', '갖다', '되다', '것', '자료형', '하다', '프로그램', '수행', '하다', '그', '프로그램', '다루다', '모든', '데이터', '메모리', '위', '저장', '되어다', '하다', '하다', '그때', '어떻다', '식', '저장', '되다', '대해', '서', '설명', '하다', '메모리', '저장', '되다', '자료', '우리', '객체', '부르다', '모든', '객체', '자료형', '갖다', '수치', '자료', '경우', '정수', '실수', '로', '구분', '하다', '또는', '문자', '데이터', '올라가다', '하다', '그래서', '각각', '대해', '서', '우리', '자료', '갖다', '붙이', '게', '되다', '모든', '개체', '고유', '갖다', '하다', '객체', '저장', '하다', '공간', '변수', '하다', '그', '공간', '이름', '변수', '명', '하다', '그', '이름', '통해', '서', '객체', '접근', '하다', '수가', '있다', '간단하다', '아래', '객체', '변수', '해해', '보다', '하다', '데이터', '만들다', '즉', '객체', '만들다', '파이썬', '어떻다', '이름', '하다', '값', '적다', '주다', '하다', '예', '들어서다', '에이', '100', '하다', '에이', '백', '갖다', '뜻', '늘다', '같다', '뜻', '아니다', '100', '값', '에이', '넣다', '이야기', '되다', '그래서', '100', '값', '에이', '들어가다', '되다', '때', '내', '부적', '일어나다', '일', '100', '데이터'

['입력', '되다', '데이터', '숫자', '인', '경우', '수치', '데이터', '하다', '문자', '인', '경우', '문자', '데이터', '하다', '다시', '말', '하다', '입력', '되다', '데이터', '각자', '데이터', '유형', '갖다', '되다', '것', '자료형', '하다', '프로그램', '수행', '하다', '그', '프로그램', '다루다', '모든', '데이터', '메모리', '위', '저장', '되어다', '하다', '하다', '그때', '어떻다', '식', '저장', '되다', '대해', '서', '설명', '하다', '메모리', '저장', '되다', '자료', '우리', '객체', '부르다', '모든', '객체', '자료형', '갖다', '수치', '자료', '경우', '정수', '실수', '로', '구분', '하다', '또는', '문자', '데이터', '올라가다', '하다', '그래서', '각각', '대해', '서', '우리', '자료', '갖다', '붙이', '게', '되다', '모든', '개체', '고유', '갖다', '하다', '객체', '저장', '하다', '공간', '변수', '하다', '그', '공간', '이름', '변수', '명', '하다', '그', '이름', '통해', '서', '객체', '접근', '하다', '수가', '있다', '간단하다', '아래', '객체', '변수', '해해', '보다', '하다', '데이터', '만들다', '즉', '객체', '만들다', '파이썬', '어떻다', '이름', '하다', '값', '적다', '주다', '하다', '예', '들어서다', '에이', '100', '하다', '에이', '백', '갖다', '뜻', '늘다', '같다', '뜻', '아니다', '100', '값', '에이', '넣다', '이야기', '되다', '그래서', '100', '값', '에이', '들어가다', '되다', '때', '내', '부적', '일어나다', '일', '100', '데이터'

['입력', '되다', '데이터', '숫자', '인', '경우', '수치', '데이터', '하다', '문자', '인', '경우', '문자', '데이터', '하다', '다시', '말', '하다', '입력', '되다', '데이터', '각자', '데이터', '유형', '갖다', '되다', '것', '자료형', '하다', '프로그램', '수행', '하다', '그', '프로그램', '다루다', '모든', '데이터', '메모리', '위', '저장', '되어다', '하다', '하다', '그때', '어떻다', '식', '저장', '되다', '대해', '서', '설명', '하다', '메모리', '저장', '되다', '자료', '우리', '객체', '부르다', '모든', '객체', '자료형', '갖다', '수치', '자료', '경우', '정수', '실수', '로', '구분', '하다', '또는', '문자', '데이터', '올라가다', '하다', '그래서', '각각', '대해', '서', '우리', '자료', '갖다', '붙이', '게', '되다', '모든', '개체', '고유', '갖다', '하다', '객체', '저장', '하다', '공간', '변수', '하다', '그', '공간', '이름', '변수', '명', '하다', '그', '이름', '통해', '서', '객체', '접근', '하다', '수가', '있다', '간단하다', '아래', '객체', '변수', '해해', '보다', '하다', '데이터', '만들다', '즉', '객체', '만들다', '파이썬', '어떻다', '이름', '하다', '값', '적다', '주다', '하다', '예', '들어서다', '에이', '100', '하다', '에이', '백', '갖다', '뜻', '늘다', '같다', '뜻', '아니다', '100', '값', '에이', '넣다', '이야기', '되다', '그래서', '100', '값', '에이', '들어가다', '되다', '때', '내', '부적', '일어나다', '일', '100', '데이터'

['입력', '되다', '데이터', '숫자', '인', '경우', '수치', '데이터', '하다', '문자', '인', '경우', '문자', '데이터', '하다', '다시', '말', '하다', '입력', '되다', '데이터', '각자', '데이터', '유형', '갖다', '되다', '것', '자료형', '하다', '프로그램', '수행', '하다', '그', '프로그램', '다루다', '모든', '데이터', '메모리', '위', '저장', '되어다', '하다', '하다', '그때', '어떻다', '식', '저장', '되다', '대해', '서', '설명', '하다', '메모리', '저장', '되다', '자료', '우리', '객체', '부르다', '모든', '객체', '자료형', '갖다', '수치', '자료', '경우', '정수', '실수', '로', '구분', '하다', '또는', '문자', '데이터', '올라가다', '하다', '그래서', '각각', '대해', '서', '우리', '자료', '갖다', '붙이', '게', '되다', '모든', '개체', '고유', '갖다', '하다', '객체', '저장', '하다', '공간', '변수', '하다', '그', '공간', '이름', '변수', '명', '하다', '그', '이름', '통해', '서', '객체', '접근', '하다', '수가', '있다', '간단하다', '아래', '객체', '변수', '해해', '보다', '하다', '데이터', '만들다', '즉', '객체', '만들다', '파이썬', '어떻다', '이름', '하다', '값', '적다', '주다', '하다', '예', '들어서다', '에이', '100', '하다', '에이', '백', '갖다', '뜻', '늘다', '같다', '뜻', '아니다', '100', '값', '에이', '넣다', '이야기', '되다', '그래서', '100', '값', '에이', '들어가다', '되다', '때', '내', '부적', '일어나다', '일', '100', '데이터'

['입력', '되다', '데이터', '숫자', '인', '경우', '수치', '데이터', '하다', '문자', '인', '경우', '문자', '데이터', '하다', '다시', '말', '하다', '입력', '되다', '데이터', '각자', '데이터', '유형', '갖다', '되다', '것', '자료형', '하다', '프로그램', '수행', '하다', '그', '프로그램', '다루다', '모든', '데이터', '메모리', '위', '저장', '되어다', '하다', '하다', '그때', '어떻다', '식', '저장', '되다', '대해', '서', '설명', '하다', '메모리', '저장', '되다', '자료', '우리', '객체', '부르다', '모든', '객체', '자료형', '갖다', '수치', '자료', '경우', '정수', '실수', '로', '구분', '하다', '또는', '문자', '데이터', '올라가다', '하다', '그래서', '각각', '대해', '서', '우리', '자료', '갖다', '붙이', '게', '되다', '모든', '개체', '고유', '갖다', '하다', '객체', '저장', '하다', '공간', '변수', '하다', '그', '공간', '이름', '변수', '명', '하다', '그', '이름', '통해', '서', '객체', '접근', '하다', '수가', '있다', '간단하다', '아래', '객체', '변수', '해해', '보다', '하다', '데이터', '만들다', '즉', '객체', '만들다', '파이썬', '어떻다', '이름', '하다', '값', '적다', '주다', '하다', '예', '들어서다', '에이', '100', '하다', '에이', '백', '갖다', '뜻', '늘다', '같다', '뜻', '아니다', '100', '값', '에이', '넣다', '이야기', '되다', '그래서', '100', '값', '에이', '들어가다', '되다', '때', '내', '부적', '일어나다', '일', '100', '데이터'

['입력', '되다', '데이터', '숫자', '인', '경우', '수치', '데이터', '하다', '문자', '인', '경우', '문자', '데이터', '하다', '다시', '말', '하다', '입력', '되다', '데이터', '각자', '데이터', '유형', '갖다', '되다', '것', '자료형', '하다', '프로그램', '수행', '하다', '그', '프로그램', '다루다', '모든', '데이터', '메모리', '위', '저장', '되어다', '하다', '하다', '그때', '어떻다', '식', '저장', '되다', '대해', '서', '설명', '하다', '메모리', '저장', '되다', '자료', '우리', '객체', '부르다', '모든', '객체', '자료형', '갖다', '수치', '자료', '경우', '정수', '실수', '로', '구분', '하다', '또는', '문자', '데이터', '올라가다', '하다', '그래서', '각각', '대해', '서', '우리', '자료', '갖다', '붙이', '게', '되다', '모든', '개체', '고유', '갖다', '하다', '객체', '저장', '하다', '공간', '변수', '하다', '그', '공간', '이름', '변수', '명', '하다', '그', '이름', '통해', '서', '객체', '접근', '하다', '수가', '있다', '간단하다', '아래', '객체', '변수', '해해', '보다', '하다', '데이터', '만들다', '즉', '객체', '만들다', '파이썬', '어떻다', '이름', '하다', '값', '적다', '주다', '하다', '예', '들어서다', '에이', '100', '하다', '에이', '백', '갖다', '뜻', '늘다', '같다', '뜻', '아니다', '100', '값', '에이', '넣다', '이야기', '되다', '그래서', '100', '값', '에이', '들어가다', '되다', '때', '내', '부적', '일어나다', '일', '100', '데이터'

['입력', '되다', '데이터', '숫자', '인', '경우', '수치', '데이터', '하다', '문자', '인', '경우', '문자', '데이터', '하다', '다시', '말', '하다', '입력', '되다', '데이터', '각자', '데이터', '유형', '갖다', '되다', '것', '자료형', '하다', '프로그램', '수행', '하다', '그', '프로그램', '다루다', '모든', '데이터', '메모리', '위', '저장', '되어다', '하다', '하다', '그때', '어떻다', '식', '저장', '되다', '대해', '서', '설명', '하다', '메모리', '저장', '되다', '자료', '우리', '객체', '부르다', '모든', '객체', '자료형', '갖다', '수치', '자료', '경우', '정수', '실수', '로', '구분', '하다', '또는', '문자', '데이터', '올라가다', '하다', '그래서', '각각', '대해', '서', '우리', '자료', '갖다', '붙이', '게', '되다', '모든', '개체', '고유', '갖다', '하다', '객체', '저장', '하다', '공간', '변수', '하다', '그', '공간', '이름', '변수', '명', '하다', '그', '이름', '통해', '서', '객체', '접근', '하다', '수가', '있다', '간단하다', '아래', '객체', '변수', '해해', '보다', '하다', '데이터', '만들다', '즉', '객체', '만들다', '파이썬', '어떻다', '이름', '하다', '값', '적다', '주다', '하다', '예', '들어서다', '에이', '100', '하다', '에이', '백', '갖다', '뜻', '늘다', '같다', '뜻', '아니다', '100', '값', '에이', '넣다', '이야기', '되다', '그래서', '100', '값', '에이', '들어가다', '되다', '때', '내', '부적', '일어나다', '일', '100', '데이터'

KeyboardInterrupt: 

In [48]:
sen = ['입력', '되다', '데이터', '숫자', '인', '경우', '수치',
 '데이터', '하다', '문자', '인', '경우', '문자', '데이터', '하다', '다시', '말', '하다', 
 '입력', '되다', '데이터', '각자', '데이터', '유형', '갖다', '되다', '것', '자료형', '하다',
 '프로그램', '수행', '하다', '그', '프로그램', '다루다', '모든', '데이터', '메모리', '위', 
 '저장', '되어다', '하다', '하다', '그때', '어떻다', '식', '저장', '되다', '대해', '서', '설명',
 '하다', '메모리', '저장', '되다', '자료', '우리', '객체', '부르다', '모든', '객체', '자료형',
 '갖다', '수치', '자료', '경우', '정수', '실수', '로', '구분', '하다', '또는', '문자', '데이터', 
 '올라가다', '하다', '그래서', '각각', '대해', '서', '우리', '자료', '갖다', '붙이', '게', '되다', 
 '모든', '개체', '고유', '갖다', '하다', '객체', '저장', '하다', '공간', '변수', '하다', '그', '공간',
 '이름', '변수', '명', '하다', '그', '이름', '통해', '서', '객체', '접근', '하다', '수가', '있다', 
 '간단하다', '아래', '객체', '변수', '해해', '보다', '하다', '데이터', '만들다', '즉', '객체', 
 '만들다', '파이썬', '어떻다', '이름', '하다', '값', '적다', '주다', '하다', '예', '들어서다', 
 '에이', '100', '하다', '에이', '백', '갖다', '뜻', '늘다', '같다', '뜻', '아니다', '100', '값', 
 '에이', '넣다', '이야기', '되다', '그래서', '100', '값', '에이', '들어가다', '되다', '때', '내', 
 '부적', '일어나다', '일', '100', '데이터', '생기', '그', '100', '데이터', '객체', '부르다', 
 '객체', '저장', '되다', '저장', '되다', '공간', '우리', '변수', '부르다', '리다', '빨갛다', 
 '박스', '바로', '변수', '되다', '변수', '이름', '하나', '씩', '가다', '붙다', '되다', '그', 
 '이름', '변수', '명', '하다', '여기', '변수', '명', '가다', '붙다', '예제', '그래서', '다음', 
 '부터', '100', '데이터', '가지', '고', '오다', '변수', '명', '에이', '이용', '하다', '가져오다', '되다']
set(sen) # unique 중복제거 -> list를 set 형식 변환 

{'100',
 '가다',
 '가져오다',
 '가지',
 '각각',
 '각자',
 '간단하다',
 '값',
 '갖다',
 '같다',
 '개체',
 '객체',
 '것',
 '게',
 '경우',
 '고',
 '고유',
 '공간',
 '구분',
 '그',
 '그때',
 '그래서',
 '내',
 '넣다',
 '늘다',
 '다루다',
 '다시',
 '다음',
 '대해',
 '데이터',
 '되다',
 '되어다',
 '들어가다',
 '들어서다',
 '때',
 '또는',
 '뜻',
 '로',
 '리다',
 '만들다',
 '말',
 '메모리',
 '명',
 '모든',
 '문자',
 '바로',
 '박스',
 '백',
 '변수',
 '보다',
 '부르다',
 '부적',
 '부터',
 '붙다',
 '붙이',
 '빨갛다',
 '생기',
 '서',
 '설명',
 '수가',
 '수치',
 '수행',
 '숫자',
 '식',
 '실수',
 '씩',
 '아니다',
 '아래',
 '어떻다',
 '에이',
 '여기',
 '예',
 '예제',
 '오다',
 '올라가다',
 '우리',
 '위',
 '유형',
 '이름',
 '이야기',
 '이용',
 '인',
 '일',
 '일어나다',
 '입력',
 '있다',
 '자료',
 '자료형',
 '저장',
 '적다',
 '접근',
 '정수',
 '주다',
 '즉',
 '통해',
 '파이썬',
 '프로그램',
 '하나',
 '하다',
 '해해'}